In [1]:
from datamanager import PINFDataParser, PINFDataParserConfig
import pathlib

parser_config = PINFDataParserConfig(data=pathlib.Path("../data/ScalarReal"))
parser: PINFDataParser = parser_config.setup()
parser.get_dataparser_outputs()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
